Zebranie wszystkich graczy z ostatnich 20 lat, przy uzyciu nba_api (dok. https://github.com/swar/nba_api)

In [5]:
import time
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from nba_api.stats.endpoints import commonallplayers #informacja o graczach
from nba_api.stats.endpoints import playergamelogs #informacja o grach

In [14]:
all_player_data = commonallplayers.CommonAllPlayers().get_data_frames()[0]

filtered_players = all_player_data[all_player_data['TO_YEAR'].apply(lambda x: int(x)) > 2003]

In [15]:
filtered_players.to_csv('players.csv') #zapis pliku odbywa się w miejscu przechowywania kodu. Prośba o zapisanie kodu na dysku przed uruchomieniem

Zebranie danych statystycznych z kazdego meczu rozegranego przez zawodnikow z ostatnich 20 lat (przy pomocy wyzej wymienionego nba_api). 

!!!Przed runem ostrzegam, że lepiej nie duplikować ćwiczenia. Samo API potrafi wielokrotnie odrzucać połączenie lub zrywać je w niespodziewany sposób. Czas ciągniecia danych potrafił przekroczyć 30h w jednym przypadku, a 6h w drugim. W paczce załączone być pliki CSV z wyciągniętymi danymi.

In [ ]:
all_player_game_logs = []

for index, player_info in filtered_players.iterrows():
    player_id = player_info['PERSON_ID']
    player_name = player_info['DISPLAY_FIRST_LAST']
    
    print(f"Collecting player game logs for {player_name}...")
    
    for season_id in ['2022-23', '2021-22', '2020-21', '2019-20', '2018-19', '2017-18', '2016-17', '2015-16', '2014-15', '2013-14', '2012-13', '2011-12', '2010-11', '2009-10', '2008-09', '2007-08', '2006-07', '2005-06', '2004-05', '2003-04']:  
        while True:
            try:
                player_logs = playergamelogs.PlayerGameLogs(player_id_nullable=player_id, season_nullable=season_id).get_data_frames()[0]
                break 
            except Exception as e:
                print(f"An error occurred: {str(e)}. Retrying in 5 seconds...")
                time.sleep(5) 
        
        all_player_game_logs.append((player_name, player_logs))

result_df = pd.concat([log[1] for log in all_player_game_logs], ignore_index=True)


In [3]:
result_df.to_csv('stats.csv')

Zebranie danych dot. playoffów dla wszystkich zawodników z ostatnich 20 lat (przy pomocy wyzej wymienionego nba_api).

In [ ]:
# Pobranie player game logs dla każdego gracza z filtered_players
all_player_game_logs = []

for index, player_info in filtered_players.iterrows():
    player_id = player_info['PERSON_ID']
    player_name = player_info['DISPLAY_FIRST_LAST']
    
    # Pobranie player game logs dla danego gracza
    print(f"Collecting player game logs for {player_name}...")
    
    # Zmiana zakresu sezonów
    for season_id in ['2022-23', '2021-22', '2020-21', '2019-20', '2018-19', '2017-18', '2016-17', '2015-16', '2014-15', '2013-14', '2012-13', '2011-12', '2010-11', '2009-10', '2008-09', '2007-08', '2006-07', '2005-06', '2004-05', '2003-04']:  # Dodaj sezon_id zgodnie z Twoimi potrzebami
        while True:
            try:
                player_logs = playergamelogs.PlayerGameLogs(player_id_nullable=player_id, season_nullable=season_id, season_type_nullable='Playoffs').get_data_frames()[0]
                break  # Przerwij pętlę, jeśli pomyślnie pobrano dane
            except Exception as e:
                if 'resultSet' in str(e): 
                    break
                else:
                    print(f"An error occurred: {str(e)}. Retrying in 5 seconds...")
                    time.sleep(5)  # Poczekaj 5 sekund przed ponowną próbą
        
        # Dodanie danych gracza do listy
        all_player_game_logs.append((player_name, player_logs))

# Konwersja listy do jednego DataFrame
result_df1 = pd.concat([log[1] for log in all_player_game_logs], ignore_index=True)

In [ ]:
result_df1.to_csv('stats_playoffs.csv')

Zebranie danych dot. kontuzji zawodników z ostatnich 20 lat (przy pomocy scrappera: https://github.com/gboogy/nba-injury-data-scraper)

In [ ]:
"""
Descripiton:
    This script is a webscraper for injury data from the Pro Sports Transaction website
Written by:
    G-R-H
"""

def replace_all(text, dic):
    '''
    This function will replace characters in text given a dictionary of characters to seach for and replace
    '''
    rc = re.compile('|'.join(map(re.escape, dic)))

    def translate(match):
        return dic[match.group(0)]
    return rc.sub(translate, text)


# Dictionary of characters to remove from text
char_replace = {' • ': ''}

# Create list of records read from webscrapper
list_of_rows = []

# Loop through webpage table, scrap data, and store lists
for i in range(0, 53850, 25):
    url = 'https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2003-10-01&EndDate=&ILChkBx=yes&InjuriesChkBx=yes&Submit=Search&start={}'.format(i)
    response = requests.get(url)
    html = response.content

    soup = BeautifulSoup(html, 'lxml')

    table = soup.find('table', attrs={'class': 'datatable center'})

    for row in table.findAll('tr', attrs={'align': 'left'}):
        list_of_cells = []

        for cell in row.findAll('td'):
            text = replace_all(cell.text, char_replace)
            text = text.strip()
            list_of_cells.append(text)
        list_of_rows.append(list_of_cells)

# Store data in a dataframe for manipulation
injuries_df = pd.DataFrame(list_of_rows, columns=[
                           'Date', 'Team', 'Acquired', 'Relinquished', 'Notes'])

acq = injuries_df['Acquired']
rel = injuries_df['Relinquished']

# Remove instances where value is like "Name 1/ Name 2"
injuries_df['Acquired'] = np.where(
    acq.str.contains('/'), acq.str.split('/ ').str[1], acq)
injuries_df['Relinquished'] = np.where(
    rel.str.contains('/'), rel.str.split('/ ').str[1], rel)

# Remove instances where value is like "(some text)"
injuries_df['Acquired'] = injuries_df.Acquired.str.replace(
    r"[\(\[].*?[\)\]]", "")
injuries_df['Relinquished'] = injuries_df.Relinquished.str.replace(
    r"[\(\[].*?[\)\]]", "")

# Write contents to csv file
injuries_df.to_csv('injuries.csv', index=False)

Dane dot. kontraktów pobrane ze strony espn.com. 

Zebranie danych z commonplayersinfo na temat wzrostu zawodnika itp.

In [ ]:
all_player_info = []

for index, player_info in filtered_players.iterrows():
    player_id = player_info['PERSON_ID']
    player_name = player_info['DISPLAY_FIRST_LAST']
    
    print(f"Collecting player info for {player_name}...")
    
    while True:
        try:
            player_info_data = commonplayerinfo.CommonPlayerInfo(player_id=player_id).get_data_frames()[0]
            break
        except Exception as e:
            print(f"An error occurred while fetching player info: {str(e)}. Retrying in 5 seconds...")
            time.sleep(5) 
    
    all_player_info.append(player_info_data)

# Concatenate all player info into one DataFrame
result_df = pd.concat(all_player_info, ignore_index=True)


In [ ]:
result_df.to_csv('playersinfo.csv')